In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

In [ ]:
from smps.physics.water_balance import create_two_bucket_model
from smps.physics.pedotransfer import estimate_soil_parameters_saxton
from smps.core.config import get_config

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Create configuration
config = get_config()


In [ ]:
#  Create Soil Parameters
print("1. Creating soil parameters...")
soil_params = estimate_soil_parameters_saxton(
    sand_percent=40,
    clay_percent=20,
    organic_matter_percent=2.0
)

print(f"   Soil texture: {soil_params.sand_percent:.0f}/"
      f"{soil_params.silt_percent:.0f}/{soil_params.clay_percent:.0f}")
print(f"   Porosity: {soil_params.porosity:.3f}")
print(f"   Field capacity: {soil_params.field_capacity:.3f}")
print(f"   Wilting point: {soil_params.wilting_point:.3f}")
print(f"   K_sat: {soil_params.saturated_hydraulic_conductivity_cm_day:.1f} cm/day")


In [ ]:
# Create Water Balance Model
print("\n2. Creating two-bucket water balance model...")
model = create_two_bucket_model(soil_params)

In [ ]:
# Scenario 1: Dry Period
print("\n3. Scenario 1: Dry period (10 days no rain)")
dates = pd.date_range('2023-06-01', periods=10, freq='D')
forcings_dry = pd.DataFrame({
    'date': dates,
    'precipitation_mm': 0.0,
    'et0_mm': 5.0,  # Moderate ET
    'ndvi': 0.7,
    'irrigation_mm': 0.0
})

results_dry = model.run_period(forcings_dry, initial_date=date(2023, 6, 1))
model.reset()


In [ ]:
# Scenario 2: Rain Event
print("\n4. Scenario 2: Rain event")
dates = pd.date_range('2023-06-11', periods=10, freq='D')
forcings_rain = pd.DataFrame({
    'date': dates,
    'precipitation_mm': [0, 0, 0, 20, 5, 0, 0, 0, 0, 0],  # Rain on day 4
    'et0_mm': 5.0,
    'ndvi': 0.7,
    'irrigation_mm': 0.0
})

results_rain = model.run_period(forcings_rain, initial_date=date(2023, 6, 11))
model.reset()

In [ ]:
# Scenario 3: Irrigation Scheduling
print("\n5. Scenario 3: Irrigation scheduling")
dates = pd.date_range('2023-06-21', periods=15, freq='D')
forcings_irrig = pd.DataFrame({
    'date': dates,
    'precipitation_mm': 0.0,
    'et0_mm': 6.0,  # High ET
    'ndvi': 0.8,
    'irrigation_mm': [0, 0, 0, 0, 20, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0]  # Irrigation on days 5 and 10
})

results_irrig = model.run_period(forcings_irrig, initial_date=date(2023, 6, 21))


In [ ]:
# Plot Results
print("\n6. Generating plots...")

fig, axes = plt.subplots(3, 2, figsize=(14, 12))
fig.suptitle('Physics Prior Model Demonstration', fontsize=16, fontweight='bold')

In [ ]:
# Plot 1: Dry period
ax = axes[0, 0]
ax.plot(results_dry.index, results_dry['theta_phys_surface'], 
        label='Surface (0-10cm)', linewidth=2)
ax.plot(results_dry.index, results_dry['theta_phys_root'], 
        label='Root zone (10-40cm)', linewidth=2)
ax.axhline(soil_params.field_capacity, color='green', linestyle='--', 
          label='Field capacity', alpha=0.7)
ax.axhline(soil_params.wilting_point, color='red', linestyle='--', 
          label='Wilting point', alpha=0.7)
ax.set_title('Scenario 1: Dry Period', fontweight='bold')
ax.set_ylabel('Soil moisture (VWC)')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)

In [ ]:
#Plot 2: ET components during dry period
ax = axes[0, 1]
et_components = results_dry[['flux_evaporation_mm', 'flux_transpiration_mm']]
et_components.plot.area(ax=ax, alpha=0.7)
ax.set_title('ET Components During Dry Period', fontweight='bold')
ax.set_ylabel('ET (mm/day)')
ax.set_xlabel('Date')
ax.legend(['Evaporation', 'Transpiration'])
ax.grid(True, alpha=0.3)

In [ ]:
# Plot 3: Rain event
ax = axes[1, 0]
ax.plot(results_rain.index, results_rain['theta_phys_surface'], 
        label='Surface', linewidth=2)
ax.plot(results_rain.index, results_rain['theta_phys_root'], 
        label='Root zone', linewidth=2)
ax.axhline(soil_params.field_capacity, color='green', linestyle='--', alpha=0.7)
ax.axhline(soil_params.wilting_point, color='red', linestyle='--', alpha=0.7)

# Add precipitation bars
ax2 = ax.twinx()
ax2.bar(results_rain.index, forcings_rain['precipitation_mm'], 
        alpha=0.3, color='blue', width=0.5, label='Precipitation')
ax2.set_ylabel('Precipitation (mm)', color='blue')
ax2.tick_params(axis='y', labelcolor='blue')

ax.set_title('Scenario 2: Rain Event', fontweight='bold')
ax.set_ylabel('Soil moisture (VWC)')
ax.set_xlabel('Date')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

In [ ]:
# Plot 4: Fluxes during rain event
ax = axes[1, 1]
fluxes = results_rain[['flux_infiltration_mm', 'flux_runoff_mm', 
                       'flux_percolation_mm', 'flux_drainage_mm']]
fluxes.plot(ax=ax, linewidth=2)
ax.set_title('Water Fluxes During Rain Event', fontweight='bold')
ax.set_ylabel('Flux (mm/day)')
ax.set_xlabel('Date')
ax.legend(['Infiltration', 'Runoff', 'Percolation', 'Drainage'])
ax.grid(True, alpha=0.3)

In [ ]:
# Plot 5: Irrigation scheduling
ax = axes[2, 0]
ax.plot(results_irrig.index, results_irrig['theta_phys_root'], 
        label='Root zone moisture', linewidth=3, color='darkgreen')
ax.axhline(soil_params.field_capacity, color='green', linestyle='--', 
          label='Field capacity', alpha=0.7)
ax.axhline(soil_params.field_capacity * 0.7, color='orange', linestyle='--', 
          label='Irrigation threshold (70% FC)', alpha=0.7)
ax.axhline(soil_params.wilting_point, color='red', linestyle='--', 
          label='Wilting point', alpha=0.7)


# Add irrigation events
irrigation_days = forcings_irrig[forcings_irrig['irrigation_mm'] > 0].index
for day in irrigation_days:
    ax.axvline(day, color='blue', linestyle=':', alpha=0.7)
    ax.text(day, soil_params.wilting_point, 'Irrigation', 
            rotation=90, verticalalignment='bottom', fontsize=8)

ax.set_title('Scenario 3: Irrigation Scheduling', fontweight='bold')
ax.set_ylabel('Root zone moisture (VWC)')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)

In [ ]:
# Plot 6: Water balance error
ax = axes[2, 1]
all_results = pd.concat([results_dry, results_rain, results_irrig])
ax.plot(all_results.index, all_results['water_balance_error_mm'].abs(), 
        linewidth=2, color='purple')
ax.axhline(1.0, color='red', linestyle='--', label='1 mm tolerance')
ax.set_title('Water Balance Error', fontweight='bold')
ax.set_ylabel('Absolute error (mm)')
ax.set_xlabel('Date')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/physics_prior_demonstration.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Print Summary Statistics
print("\n7. Summary Statistics")
print("=" * 50)

all_results = pd.concat([results_dry, results_rain, results_irrig])

print(f"Simulation period: {len(all_results)} days")
print(f"\nSoil moisture statistics:")
print(f"  Surface - Mean: {all_results['theta_phys_surface'].mean():.3f}")
print(f"           Min: {all_results['theta_phys_surface'].min():.3f}")
print(f"           Max: {all_results['theta_phys_surface'].max():.3f}")
print(f"  Root zone - Mean: {all_results['theta_phys_root'].mean():.3f}")
print(f"             Min: {all_results['theta_phys_root'].min():.3f}")
print(f"             Max: {all_results['theta_phys_root'].max():.3f}")

print(f"\nWater balance performance:")
print(f"  Mean absolute error: {all_results['water_balance_error_mm'].abs().mean():.3f} mm")
print(f"  Max error: {all_results['water_balance_error_mm'].abs().max():.3f} mm")
print(f"  Days with error > 1mm: {(all_results['water_balance_error_mm'].abs() > 1).sum()}")

print(f"\nET partitioning:")
print(f"  Mean evaporation: {all_results['flux_evaporation_mm'].mean():.1f} mm/day")
print(f"  Mean transpiration: {all_results['flux_transpiration_mm'].mean():.1f} mm/day")
print(f"  Mean ET: {all_results['flux_evapotranspiration_mm'].mean():.1f} mm/day")

In [ ]:
#  Test with different soil textures
print("\n8. Soil texture sensitivity analysis")
print("=" * 50)

textures = [
    ("Sand", 85, 5),
    ("Loam", 40, 20),
    ("Clay", 20, 40)
]

sensitivity_results = []

for texture_name, sand, clay in textures:
    soil = estimate_soil_parameters_saxton(sand, clay)
    model = create_two_bucket_model(soil)
    
    # Run standard scenario
    results = model.run_period(forcings_rain, initial_date=date(2023, 6, 11))
    
    sensitivity_results.append({
        'texture': texture_name,
        'sand': sand,
        'clay': clay,
        'mean_surface_moisture': results['theta_phys_surface'].mean(),
        'mean_root_moisture': results['theta_phys_root'].mean(),
        'total_runoff': results['flux_runoff_mm'].sum(),
        'total_drainage': results['flux_drainage_mm'].sum()
    })

sensitivity_df = pd.DataFrame(sensitivity_results)
print(sensitivity_df.to_string(index=False))

print("\n✅ Physics prior model demonstration complete!")